# Generate IL expert data for behavior cloning
Given a number of historical scenarios containing real-time price and solar PV generation data, run MILP on each scenario to get the optimal solutions of load management. Then, extract the MILP expert's state-action dataset.

In [10]:
using DataFrames, CSV
using BSON: @save, @load
using BSON, NPZ
using TOML
using JuMP
using Revise, HEMS

ENV["COLUMNS"] = 2000;

## Read raw scenarios data and home configuration

In [11]:
root_dir = "../../"

"../../"

In [12]:
# home configuration
home_dir = joinpath(root_dir, "data/home/")
h = read_home_config(joinpath(home_dir, "home.toml"))
;

In [13]:
scenario_dir = joinpath(root_dir, "data/learning/scenarios/")

"../../data/learning/scenarios/"

In [14]:
# three years' data: 2019 and 2020 for train and 2021 for test 
train_ss = [BSON.load(joinpath(scenario_dir, "2019.bson"))[:ss] .|> DataFrame;
    BSON.load(joinpath(scenario_dir, "2020.bson"))[:ss] .|> DataFrame]
test_ss = BSON.load(joinpath(scenario_dir, "2021.bson"))[:ss] .|> DataFrame;

## Optimize each scenario with the MILP expert

In [15]:
IL_data_dir = joinpath(root_dir, "data/learning/IL")

"../../data/learning/IL"

In [16]:
run_expert!(test_ss, h; out_bson = joinpath(IL_data_dir, "test_ss.bson"))
# also measure the training time
@time begin
    run_expert!(train_ss, h; out_bson = joinpath(IL_data_dir, "train_ss.bson"))
end
;

198.625715 seconds (114.16 M allocations: 8.100 GiB, 1.02% gc time)


In [17]:
# check the first result
first(train_ss)

,ρ,PPV,P_H,C_H,C_H_dc,P_AC1,C_AC1,C_AC1_dc,P_AC2,C_AC2,C_AC2_dc,z_WM,P_WM,C_WM,z_DW,P_DW,C_DW,z_EV,P_EV,C_EV,zb,zs,Pb,Ps,Cele,P_load,C_load,C
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0901989,0.0,2.25,0.236698,0.03375,2.125,0.208548,0.016875,1.06,0.114971,0.01936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6.135,0.0,0.55337,6.135,0.623355,0.623355
2,0.0888795,0.0,2.25,0.233729,0.03375,2.125,0.205744,0.016875,1.06,0.113572,0.01936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6.135,0.0,0.545276,6.135,0.615261,0.615261
3,0.0818565,0.0,2.25,0.217927,0.03375,2.125,0.19082,0.016875,1.1,0.106042,0.016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6.175,0.0,0.505464,6.175,0.572089,0.572089
4,0.101792,0.0,2.1,0.262363,0.0486,2.05,0.232974,0.0243,0.98,0.126796,0.02704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.83,0.0,0.593447,5.83,0.693387,0.693387
5,0.0849343,0.0,2.25,0.224852,0.03375,2.125,0.19736,0.016875,1.06,0.10939,0.01936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6.135,0.0,0.521072,6.135,0.591057,0.591057
6,0.124614,0.0,1.95,0.309147,0.06615,1.975,0.279188,0.033075,0.86,0.148128,0.04096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.485,0.0,0.683508,5.485,0.823693,0.823693
7,0.08178,0.0,2.25,0.217755,0.03375,2.125,0.190657,0.016875,1.1,0.105958,0.016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6.175,0.0,0.504991,6.175,0.571616,0.571616
8,0.0858055,0.0505899,2.25,0.226812,0.03375,2.125,0.199212,0.016875,1.06,0.110314,0.01936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6.08441,0.0,0.522076,6.135,0.596402,0.592061
9,0.0896529,0.606784,2.25,0.235469,0.03375,2.125,0.207387,0.016875,1.06,0.114392,0.01936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.52822,0.0,0.495621,6.135,0.620006,0.565606


## Build state-action dataset as expert demonstrations
- Extract X-y pairs from the optimized scenarios.\
- Write into the specific `out_bson` file.

In [18]:
test_xy = build_expert_dataset(test_ss, h; out_bson = joinpath(IL_data_dir, "test_xy.bson"), H=1)
@time begin
    train_xy = build_expert_dataset(train_ss, h; out_bson = joinpath(IL_data_dir, "train_xy.bson"), H=1)
end
;

  0.263083 seconds (1.07 M allocations: 74.712 MiB, 22.06% gc time)
